## Self-Driving Car Engineer Nanodegree
### Project: Finding Lanes on the Road

In this project I built a pipeline to analyze video clips on a frame-by-frame basis and detect the lines on the road. 
The basic pipeline follows these steps: 

- Turn color image to grey scale and apply gamma correction to enhance whites
- Apply a white color mask to grey scale image
- Apply a yellow color mask to color image
- Combine color masks
- Select a region of interest and filter out anything outside of it
- Apply a gaussian blur to the combined mask
- Perform Canny edge detection
- Apply a Hough transform to detect lines
- Interpolate and consolidate detected lines into two lines: one for the left side of the road and one for the right

### Python Code
Import relevant libraries:

In [1]:
from moviepy.editor import VideoFileClip
import numpy as np
import cv2
import math

### Color Selection

In [2]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

def mask_white(image):
    [ret, thresholded] = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY)
    return thresholded

def mask_yellow(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    lower = np.array((15,60,100))
    upper = np.array((40,255,255))
    yellow_mask = cv2.inRange(hsv, lower, upper)
    return cv2.bitwise_and(image, image, mask = yellow_mask)

### Roi Selection

In [3]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

### Gaussian Blur

In [4]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

### Edge Detection

In [5]:
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

### Line Detection and Interpolation
Line interpolation is performed by calculating the slope and x and y axes intercepts. The lines are classified according to their slope (positive or negative) constrained to having the intercept with the x axis (on the bottom of the image) at the correct half of the image (left half for left line, right half for right line). Lastly, a weighted average is performed on the two groups, using the length of the line as its weight. 

In [11]:
def average_lines(lines, imwidth, imheight):
    
    left_lines    = [] # (slope, yintercept, length)
    right_lines   = [] # (slope, yintercept, length)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2==x1:
                continue # ignore a vertical line
            
            slope = (y2-y1)/(x2-x1)
            
            if abs(slope) < 0.5:
                continue   # ignore slopes smaller than 0.5
                
            yintercept = y1 - slope*x1
            xintercept = (imheight-yintercept) / slope   # intercept of the line with the bottom of the image
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)
            
            if length > 10:     # discard small segments 
                if slope < 0 and xintercept >=0 and xintercept <= imwidth/2: 
                    left_lines.append([slope, yintercept, length])
                elif slope > 0 and xintercept >= imwidth/2 and xintercept <= imwidth:
                    right_lines.append([slope, yintercept, length])
    
    if len(left_lines) > 0:
        left_lines = np.array(left_lines)
        left_idx = get_valid_indexes(left_lines[:, 0], std=1.5)
        left_lines = left_lines[left_idx, :]
        left_weights = left_lines[:, 2]
    else:
        left_weights = []
    
    if len(right_lines) > 0:
        right_lines = np.array(right_lines)
        right_idx = get_valid_indexes(right_lines[:, 0], std=1.5)
        right_lines = right_lines[right_idx, :]
        right_weights = right_lines[:, 2]
    else:
        right_weights = []
    
    # weighted average of lines that uses the length of the lines as a weight   
    left_lane  = np.dot(left_weights,  left_lines[:, 0:2]) /np.sum(left_weights)  if len(left_weights) >0 else None
    right_lane = np.dot(right_weights, right_lines[:, 0:2])/np.sum(right_weights) if len(right_weights)>0 else None
    
    return left_lane, right_lane 

def lines_to_pixels(y1, y2, line):
    
    if line is None:
        return None
    
    slope, intercept = line

    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    
    return ((x1, y1), (x2, y2))


def lane_lines(image, lines):
    
    left_lane, right_lane = average_lines(lines, image.shape[1], image.shape[0])
    
    y1 = image.shape[0] # bottom of the image
    y2 = (image.shape[0]/2)*1.2         

    left_line  = lines_to_pixels(y1, y2, left_lane)
    right_line = lines_to_pixels(y1, y2, right_lane)
    
    return left_line, right_line

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
            
    for line in lines:
        if line is not None:
            cv2.line(img, *line,  color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    newlines = lane_lines(line_img, lines)
    draw_lines(line_img, newlines)
    return line_img

### Other auxiliary functions

In [12]:
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def get_valid_indexes(observed, std=1.5):
    return np.array(abs(observed - np.mean(observed)) < std*np.std(observed))

### Applying the pipeline to videos

In [13]:
class LaneDetector(object):
    '''
    classdocs
    '''


    def __init__(self):
        '''
        Constructor
        '''
        pass
    
    def process(self, input_video, output_video):
        original = VideoFileClip(input_video)
        marked = original.fl_image(self.pipeline_for_frame) 
        marked.write_videofile(output_video, audio=False)

    def pipeline_for_frame(self, image):
        
        imshape = image.shape
        grey = grayscale(image)
        gamma_corrected = adjust_gamma(grey, 0.3)
        yellow_mask = mask_yellow(image)
        vertices = np.array([[(0,imshape[0]),(int((imshape[1]/2)*0.97), int((imshape[0]/2)*1.2)), (int((imshape[1]/2)*1.03), int((imshape[0]/2)*1.2)), (imshape[1],imshape[0])]], dtype=np.int32)
        yellow_mask_filtered = region_of_interest(yellow_mask, vertices)
    
        white_mask = mask_white(gamma_corrected)
        white_mask_filtered = region_of_interest(white_mask, vertices)
        
        thresh = 127
        [ret, yellow_mask_filtered_bw] = cv2.threshold(grayscale(yellow_mask_filtered), thresh, 255, cv2.THRESH_BINARY)
        combined_mask = weighted_img(yellow_mask_filtered_bw, white_mask_filtered, 1.0)
        
        blurred_mask = gaussian_blur(combined_mask, 7)

        canny_edges = canny(blurred_mask, 230, 250)
        
        rho = 2 # distance resolution in pixels of the Hough grid
        theta = np.pi/180 # angular resolution in radians of the Hough grid
        threshold = 5     # minimum number of votes (intersections in Hough grid cell)
        min_line_length = 20 #minimum number of pixels making up a line
        max_line_gap = 10    # maximum gap in pixels between connectable line segments
        hough_lines_image = hough_lines(canny_edges, rho, theta, threshold, min_line_length, max_line_gap)
        
        output = weighted_img(hough_lines_image, image, 0.8, 1.0)
        return output

    
import os
    
input_dir = "test_videos/"
output_dir = os.path.join(input_dir, "processed")
input_videos = os.listdir(input_dir)
    
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
        
detector = LaneDetector()
    
for v in input_videos:
    extension = v.split(".")[-1]
    if extension == 'mp4':
        in_file = os.path.join(input_dir, v)
        out_file = os.path.join(output_dir, v)
        detector.process(in_file, out_file)

[MoviePy] >>>> Building video test_videos/processed/solidWhiteRight.mp4
[MoviePy] Writing video test_videos/processed/solidWhiteRight.mp4




  0%|          | 0/222 [00:00<?, ?it/s]

  3%|▎         | 7/222 [00:00<00:03, 63.43it/s]

  6%|▋         | 14/222 [00:00<00:03, 64.58it/s]

 10%|▉         | 22/222 [00:00<00:03, 65.76it/s]

 13%|█▎        | 29/222 [00:00<00:02, 66.58it/s]

 16%|█▌        | 36/222 [00:00<00:02, 67.08it/s]

 19%|█▉        | 43/222 [00:00<00:02, 67.56it/s]

 23%|██▎       | 50/222 [00:00<00:02, 63.99it/s]

 25%|██▌       | 56/222 [00:00<00:02, 58.77it/s]

 28%|██▊       | 62/222 [00:00<00:02, 57.10it/s]

 31%|███       | 68/222 [00:01<00:02, 56.50it/s]

 33%|███▎      | 74/222 [00:01<00:02, 56.81it/s]

 36%|███▌      | 80/222 [00:01<00:02, 53.91it/s]

 39%|███▊      | 86/222 [00:01<00:02, 55.14it/s]

 41%|████▏     | 92/222 [00:01<00:02, 55.14it/s]

 44%|████▍     | 98/222 [00:01<00:02, 54.96it/s]

 47%|████▋     | 104/222 [00:01<00:02, 54.69it/s]

 50%|████▉     | 110/222 [00:01<00:02, 55.00it/s]

 52%|█████▏    | 116/222 [00:01<00:01, 56.18it/s]

 55%|█████▍    | 122/222 [00:02<00:01, 55.73it/s]

 58%

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/processed/solidWhiteRight.mp4 

[MoviePy] >>>> Building video test_videos/processed/solidYellowLeft.mp4
[MoviePy] Writing video test_videos/processed/solidYellowLeft.mp4




  0%|          | 0/682 [00:00<?, ?it/s]

  1%|          | 6/682 [00:00<00:11, 57.68it/s]

  2%|▏         | 14/682 [00:00<00:10, 61.22it/s]

  3%|▎         | 22/682 [00:00<00:10, 63.63it/s]

  4%|▍         | 30/682 [00:00<00:09, 65.89it/s]

  6%|▌         | 38/682 [00:00<00:09, 67.68it/s]

  7%|▋         | 46/682 [00:00<00:09, 68.38it/s]

  8%|▊         | 53/682 [00:00<00:09, 63.09it/s]

  9%|▊         | 59/682 [00:00<00:10, 61.05it/s]

 10%|▉         | 65/682 [00:01<00:10, 59.32it/s]

 10%|█         | 71/682 [00:01<00:10, 57.22it/s]

 11%|█▏        | 77/682 [00:01<00:10, 55.36it/s]

 12%|█▏        | 83/682 [00:01<00:10, 56.61it/s]

 13%|█▎        | 89/682 [00:01<00:10, 56.64it/s]

 14%|█▍        | 95/682 [00:01<00:10, 57.05it/s]

 15%|█▍        | 102/682 [00:01<00:10, 57.90it/s]

 16%|█▌        | 108/682 [00:01<00:09, 58.16it/s]

 17%|█▋        | 114/682 [00:01<00:09, 58.39it/s]

 18%|█▊        | 121/682 [00:01<00:09, 58.19it/s]

 19%|█▊        | 127/682 [00:02<00:09, 57.15it/s]

 20

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/processed/solidYellowLeft.mp4 

